In [38]:
import pandas as pd


# clients = pd.read_csv("datos/caso_2/clients.csv")
# depots = pd.read_csv("datos/caso_2/depots.csv")
# depots = depots.drop(columns=["DepotID"])
# vehicles = pd.read_csv("datos/caso_2/vehicles.csv")

clients = pd.read_csv("datos/caso_2_petite/clients.csv")
depots = pd.read_csv("datos/caso_2_petite/depots.csv")
depots = depots.drop(columns=["DepotID"])
vehicles = pd.read_csv("datos/caso_2_petite/vehicles.csv")



print(clients.head())
print(depots.head())
print(vehicles.head())


   LocationID  ClientID   Latitude  Longitude  Demand   TimeWindow
0           2         1  12.341057 -71.723837    25.0  13:20-13:50
1           3         2  12.310993 -71.719252     9.0  13:30-14:00
2           4         3  12.326781 -71.668190    18.0  10:15-10:45
3           5         4  12.330562 -71.694530    22.0  13:00-13:30
4           6         5  12.346781 -71.708920    13.0  14:50-15:20
   LocationID  Latitude  Longitude
0           1  12.31846 -71.716203
   VehicleID   Type  Capacity  Range  Speed
0          1    4x4     200.0  804.0    NaN
1          2  drone      25.0  480.0  135.0


In [39]:
from haversine import haversine


def dehhmm_a_minutos(hhmm):
    """
    Convierte un tiempo en formato de horas y minutos (HHMM) a minutos.
    :param dehhmm: Tiempo en formato HHMM.
    :return: Tiempo en minutos.
    """
    horas = hhmm // 100
    minutos = hhmm % 100
    return horas * 60 + minutos


# recibe "hh:mm-hh:mm" y devuelve una lista de tuplas con los tiempos en minutos
def convertir_hora_a_minutos(rango_horas):
    """
    Convierte un rango de horas en formato "hh:mm-hh:mm" a una lista de tuplas con los tiempos en minutos.
    :param rango_horas: Rango de horas en formato "hh:mm-hh:mm".
    :return: tiempos en minutos.
    """
    inicio, fin = rango_horas.split('-')
    inicio_min = dehhmm_a_minutos(int(inicio.replace(':', '')))
    fin_min = dehhmm_a_minutos(int(fin.replace(':', '')))
    return [inicio_min, fin_min]

# haversine

def haversine_distance(coord1, coord2):
    """
    Calculate the haversine distance between two coordinates.
    :param coord1: Tuple of (latitude, longitude) for the first coordinate.
    :param coord2: Tuple of (latitude, longitude) for the second coordinate.
    :return: Distance in kilometers.
    """
    return haversine(coord1, coord2)

In [40]:

depots = depots.rename(columns={"LocationID": "id"})
clients = clients.rename(columns={"LocationID": "id"})
depots["id"] = 0 
depots["Demand"] = 0

clients = clients.reset_index(drop=True)
clients["id"] = clients.index + 1  

depots = depots[["id", "Longitude", "Latitude", "Demand"]]
clients = clients[["id", "Longitude", "Latitude", "Demand"]]

nodes = pd.concat([depots, clients], ignore_index=True)


N = nodes["id"].tolist()

C = clients["id"].tolist()

De = nodes.set_index("id")["Demand"].to_dict()

depot = 0

V = vehicles["VehicleID"].tolist()
P = vehicles.set_index("VehicleID")["Capacity"].to_dict()
M = vehicles.set_index("VehicleID")["Range"].to_dict()


distances = {}

for i in range(len(nodes)):
    for j in range(len(nodes)):
        if i != j:
            coord1 = (nodes.iloc[i]["Latitude"], nodes.iloc[i]["Longitude"])
            coord2 = (nodes.iloc[j]["Latitude"], nodes.iloc[j]["Longitude"])
            distances[(nodes.iloc[i]["id"], nodes.iloc[j]["id"])] = haversine_distance(coord1, coord2) *1.3 # 30% más por ser un carro

print("Nodos:", N)
print("Clientes:", C)
print("Vehículos:", V)
print("Capacidades:", P)
print("Alcances:", M)
print("Demanda:", De)



Nodos: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Clientes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Vehículos: [1, 2]
Capacidades: {1: 200.0, 2: 25.0}
Alcances: {1: 804.0, 2: 480.0}
Demanda: {0: 0.0, 1: 25.0, 2: 9.0, 3: 18.0, 4: 22.0, 5: 13.0, 6: 4.0, 7: 5.0, 8: 10.0, 9: 9.0, 10: 9.0, 11: 10.0, 12: 24.0, 13: 12.0, 14: 15.0}


In [41]:
print(distances)

{(np.float64(0.0), np.float64(1.0)): 3.439820297737471, (np.float64(0.0), np.float64(2.0)): 1.1621579203687056, (np.float64(0.0), np.float64(3.0)): 6.8863931713672075, (np.float64(0.0), np.float64(4.0)): 3.525329536814699, (np.float64(0.0), np.float64(5.0)): 4.221074201758693, (np.float64(0.0), np.float64(6.0)): 3.708981644905904, (np.float64(0.0), np.float64(7.0)): 2.9721357282532423, (np.float64(0.0), np.float64(8.0)): 4.81093076951533, (np.float64(0.0), np.float64(9.0)): 2.812974341701751, (np.float64(0.0), np.float64(10.0)): 3.9221448507664123, (np.float64(0.0), np.float64(11.0)): 5.318588738301463, (np.float64(0.0), np.float64(12.0)): 1.6631050074507683, (np.float64(0.0), np.float64(13.0)): 2.095389517864141, (np.float64(0.0), np.float64(14.0)): 4.820212683517015, (np.float64(1.0), np.float64(0.0)): 3.439820297737471, (np.float64(1.0), np.float64(2.0)): 4.393866122258987, (np.float64(1.0), np.float64(3.0)): 8.1248611854134, (np.float64(1.0), np.float64(4.0)): 4.407990382966941, (n

# MODELO

In [42]:
from pyomo.environ import *

model = ConcreteModel()

costo_por_km = 0.57
cost_dict = {(i, j): distances[i, j] * costo_por_km for i in N for j in N if i != j}

model.N = Set(initialize=N)       # Nodos 
model.C = Set(initialize=C)        # Clientes
model.V = Set(initialize=V)        # Vehiculos
model.depot = Param(initialize=depot) # Deposito

model.d = Param(model.N, model.N, initialize=distances, within=NonNegativeReals) # Distancia
model.De = Param(model.N, initialize=De, within=NonNegativeReals, default=0)      # Demanda cliente
model.P = Param(model.V, initialize=P, within=NonNegativeReals)      # Capacidad vehiculo              
model.M = Param(model.V, initialize=M, within=NonNegativeReals) # autonomia vehiculo	
model.cost = Param(model.N, model.N, initialize=cost_dict, within=NonNegativeReals, default=0) # Costo por arco

model.x = Var(model.N, model.N, model.V, domain=Binary) # Variable binaria que indica si el vehiculo k viaja del nodo i al nodo j
model.c = Var(model.N, model.N, model.V, domain=NonNegativeReals) # Variable continua que indica el costo del arco (i,j) para el vehiculo k
model.u = Var(model.N, model.V, within=NonNegativeIntegers, bounds=(1, len(N))) # subtours
model.cv = Var(model.V, domain=NonNegativeReals) # carga que lleva el vehiculo k
model.ve = Var(model.V, domain=Binary) # variable binaria que indica si el vehiculo k es utilizado o no


# Objective function
# Minimize the total cost of the routes taken by all vehicles
model.obj = Objective(
    expr=sum(model.cost[i, j] * model.x[i, j, k] for k in model.V for i in model.N for j in model.N if i != j),
    sense=minimize
)


# Constraints

# Visit once constraint: each node must be visited exactly once by one vehicle
model.visit_once = ConstraintList()
for i in model.C:
    model.visit_once.add(sum(model.x[i, j, k] for j in model.N for k in model.V if i != j) == 1)

# Flujos

model.flows = ConstraintList()
for k in model.V:
    model.flows.add(sum(model.x[0, j, k] for j in model.N if j != 0) == 1)  
    model.flows.add(sum(model.x[j, 0, k] for j in model.N if j != 0) == 1)  

model.start_node = ConstraintList()
for k in model.V:
    model.start_node.add(sum(model.x[0, j, k] for j in model.N if j != 0) == model.ve[k])
    
model.end_node = ConstraintList()
for k in model.V:
    model.end_node.add(sum(model.x[j, 0, k] for j in model.N if j != 0) == model.ve[k])
    
model.flow = ConstraintList()
for k in model.V:
    for i in model.C:
        model.flow.add(sum(model.x[i, j, k] for j in model.N if i != j) ==
                       sum(model.x[j, i, k] for j in model.N if i != j))

# Subtour elimination constraints
model.mtz = ConstraintList()
for k in model.V:
    for i in model.C:
        for j in model.C:
            if i != j:
                model.mtz.add(model.u[i, k] - model.u[j, k] + len(N) * model.x[i, j, k] <= len(N) - 1)

# Capacity constraints
model.Carga = ConstraintList()
for k in model.V:
    expr = sum(model.x[i, j, k] * model.De[j] for i in model.N for j in model.N if i != j)
    model.Carga.add(expr <= model.P[k])


# Distance constraints
model.Distancia = ConstraintList()
for k in model.V:
    expr = sum(model.x[i, j, k] * model.d[i, j] for i in model.N for j in model.N if i != j)
    model.Distancia.add(expr <= model.M[k])



In [43]:
from amplpy import modules
import pyomo.environ as pyo
# solver_name = "highs"  # "highs", "cbc",  "couenne", "bonmin", "ipopt", "scip", or "gcg".
# solver = pyo.SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")


# solver.solve(model, tee=True,options = {
#     "time_limit": 10,
#     "presolve": "on",               # on/off
#     "mip_rel_gap": 0.01,            # 1% optimality gap
#     "mip_abs_gap": 1.0,             # absolute optimality gap
#     "random_seed": 42,              # reproducibility
#     "parallel": "on",               # parallel processing (default: on)
#     "threads": 8,                   # Usa múltiples hilos (si está disponible)

# })

# model.display()

solver = SolverFactory('gurobi')
results = solver.solve(
    model,
    tee=True,
    options={
        'TimeLimit': 30,
        'MIPGap': 0.01,
        'Threads': 4,
        'Presolve': 2,
        'Cuts': 2,
        'Heuristics': 0.5
    }
)

model.display()
print("Status:", results.solver.termination_condition)
print("Objective value:", model.obj())
print("Vehicle usage:")
for k in model.V:
    if model.ve[k].value > 0:
        print(f"Vehicle {k} is used.")
    else:
        print(f"Vehicle {k} is not used.")


Read LP format model from file C:\Users\diego\AppData\Local\Temp\tmp636ugvdd.pyomo.lp
Reading time = 0.02 seconds
x1: 418 rows, 450 columns, 3196 nonzeros
Set parameter TimeLimit to value 30
Set parameter MIPGap to value 0.01
Set parameter Threads to value 4
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Set parameter Heuristics to value 0.5
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  30
MIPGap  0.01
Heuristics  0.5
Cuts  2
Presolve  2
Threads  4

Optimize a model with 418 rows, 450 columns and 3196 nonzeros
Model fingerprint: 0x2ac9a5f0
Variable types: 0 continuous, 450 integer (422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [7e-01, 6e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range 